#### Task 3
### TF-IDF, Logistic Regression
- [x] Реализовать tf-idf (sklearn)
- [x] Реализовать логистическую регрессию (sklearn)

*Davletyarov Ildar, 11-808*

In [1]:
import nltk

# stab for download nltk packages without SSL
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to /Users/ildar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from collections import defaultdict
import string
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.probability import FreqDist
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package wordnet to /Users/ildar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ildar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load data
DATA_PATH = "/Users/ildar/Desktop/labeledTrainData.tsv"
df = pd.read_csv(DATA_PATH, sep='\t', encoding='utf-8')

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...


In [5]:
df.to_dict('records')[0]

{'id': '5814_8',
 'sentiment': 1,
 'review': "With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />Th

In [6]:
COUNT_WORDS = 500
texts = df.review.tolist()
sentiment = df.sentiment.tolist()
texts = texts[:COUNT_WORDS]

In [7]:
def delete_html(text):
    return text.replace('<br />', '')

def transform_text(text, spec_chars, lemmatizer):
    frequency = {}
    w = []
    text = text.lower()
    word_list = nltk.word_tokenize(text)
    text = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    text = "".join([ch for ch in text if ch not in spec_chars])
    text = delete_html(text)
    fdist = FreqDist(text)
    freq = fdist.most_common(10)
    for j in range(10):
        if freq[j][1] >= 4:
            w.append(freq[j][0])
    if len(w) > 0:
        for k in range(len(w)):
            text.replace(w[k], '')
    for word in text:
        count = frequency.get(word,0)
        frequency[word] = count + 1
    for word in frequency.keys():
        text = text.replace(word, str(frequency[word]))
    return text

In [8]:
class NaiveBayesClassifier:
    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.class_probs = {}
        self.conditional_probabilities = {}
        self._class_freq = defaultdict(lambda: 0)
        self._feat_freq = defaultdict(lambda: 0)
    
    def fit(self, X, y):
        for feature, label in zip(X, y):
            self._class_freq[label] += 1
            for value in feature:
                self._feat_freq[(value, label)] += 1
        
        num_samples = len(X)
        for k in self._class_freq:
            self._class_freq[k] /= num_samples
            
        for value, label in self._feat_freq:
            self._feat_freq[(value, label)] /= self._class_freq[label]
            
        return self       
        
    def predict(self, X):
        return min(self._class_freq.keys(), key=lambda c : self._calculate_class_freq(X, c))
    
    def _calculate_class_freq(self, X, clss):
        freq = -np.log(self._class_freq[clss])
        for feat in X:
            freq += -np.log(self._feat_freq.get((feat, clss), 10 ** (-7)))
        return freq

In [9]:
texts[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [14]:
# tf-idf
tfidf_vectorizer = TfidfVectorizer()
values = tfidf_vectorizer.fit_transform(texts)

feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(values.toarray(), columns = feature_names)


,000,007,04,10,100,101,108,11,110,112,...,zoey,zombi,zombie,zombies,zone,zoom,zooms,zuniga,zurich,êxtase
0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.049854,0.0,0.0,0.027388,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
spec_chars = string.punctuation + '\n\t'
lemmatizer = WordNetLemmatizer()
for i in range(len(texts)):
    texts[i] = '0.' + transform_text(texts[i], spec_chars, lemmatizer) # normalize text
    texts[i] = [float(texts[i])]

In [16]:
x = np.array(texts)
y = np.array(sentiment[:COUNT_WORDS])

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

In [18]:
model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(X_train, Y_train)

LogisticRegression(random_state=0, solver='liblinear')

In [19]:
print(classification_report(Y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.49      1.00      0.66        74
           1       0.00      0.00      0.00        76

    accuracy                           0.49       150
   macro avg       0.25      0.50      0.33       150
weighted avg       0.24      0.49      0.33       150

/Users/ildar/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ildar/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ildar/Li